In [1]:
import sys
sys.path.append('../')
import joblib
import requests
import streamlit as st
import tensorflow as tf
from types import SimpleNamespace
import wikipedia 
import openai
import gradio as gr
import torch
from transformers import pipeline
from transformers import AutoTokenizer
from web.model import load_model
import importlib
from tools import ggsearch as gg
from tools.ggsearch import GoogleSearch,find_most_similar_documents

d:\Other\Desktop\Question_Answering\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-30 14:32:54.810 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
d:\Other\Desktop\Question_Answering\myenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')

In [3]:
model = load_model('../model/qa_large_model/')

2024-05-30 14:33:01.813 
  command:

    streamlit run d:\Other\Desktop\Question_Answering\myenv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-05-30 14:33:01.813 No runtime found, using MemoryCacheStorageManager


In [4]:
torch.inference_mode()
def predict(model, tokenizer, context, question):
    inputs = tokenizer.encode_plus(
        question, context, 
        return_tensors='pt'
    ).to('cpu')
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    answer_start = torch.argmax(outputs[0])  
    answer_end = torch.argmax(outputs[1]) + 1 

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(
            inputs.input_ids[0][answer_start:answer_end]
        )
    )
    
    print(
        f'- Question: {question}\n'
        f'- Answer: {answer}'
    )
    
    return question,answer


In [9]:
question = "Quốc kỳ Việt Nam có màu gì?"
context = ""

In [10]:
ggsearch = GoogleSearch()
document_urls, ggsearch_results = ggsearch.gg_search(question)
most_similar_document = find_most_similar_documents(question, ggsearch_results)
context += most_similar_document[:500]

[' Ý nghĩa của lá cờ Việt Nam là gì? Hành vi xúc phạm đến lá cờ Việt Nam bị xử lý thế nào? (Hình từ internet) Về vấn đề này, THƯ VIỆN PHÁP LUẬT giải đáp như sau: Lá cờ Việt Nam là Quốc kỳ của đất nước Việt Nam. Theo đó, tại khoản 1 Điều 13  Hiến pháp 2013 quy định Quốc kỳ nước Cộng hòa xã hội chủ nghĩa Việt Nam hình chữ nhật, chiều rộng bằng hai phần ba chiều dài, nền đỏ, ở giữa có ngôi sao vàng năm cánh. Ý nghĩa của lá cờ với nền màu đỏ tượng trưng cho màu của cách mạng, máu của các anh hùng, ngôi sao vàng tượng trưng cho linh hồn dân tộc và năm cánh sao tượng trưng cho năm tầng lớp tham gia cách mạng sĩ, nông, công, thương, binh cùng đoàn kết kháng chiến. Tại Điều lệ 974 - TTg năm 1956 quy định về việc dùng quốc kỳ nước Việt Nam dân chủ cộng hòa như sau: 2.1 Thời gian treo cơ Việt Nam Treo riêng quốc kỳ của ta - Quốc kỳ được treo trong các phòng họp của các cấp chính quyền và các đoàn thể khi họp những buổi long trọng. - Quốc kỳ chỉ treo ngoài nhà những ngày tết và ngày lễ sau đây: +

In [11]:
print(f"Question: {question}")
print(f"Context: {context}")

Question: Quốc kỳ Việt Nam có màu gì?
Context: Quốc kỳ Việt Nam Dân chủ Cộng hòa (hay còn gọi là Cờ sao mai hoặc Cờ đỏ sao vàng) là quốc kỳ của Việt Nam Dân chủ Cộng hòa một cách chính thức khi Kỳ họp thứ nhất của Quốc hội khóa I ngày 05 tháng 01 năm 1946 biểu quyết thông qua và được Hiến pháp năm 1946 ghi nhận từ ngày 09 tháng 11 năm 1946. Vào ngày 02 tháng 7 năm 1976 Quốc hội thống nhất sau Hiệp Thương tổng tuyển cử năm 1976 giữa Việt Nam Dân chủ Cộng hòa và Cộng hòa miền Nam Việt Nam xác định đây là quốc kỳ chính thức của nước Cộng hòa Xã


In [12]:
question, answer = predict(model, tokenizer, context, question)

- Question: Quốc kỳ Việt Nam có màu gì?
- Answer: Cờ sao mai hoặc Cờ đỏ sao vàng)
